# Session 3 - Part 1
This interactive notebook demonstrates how to predict retinotopic maps from anatomical data using a geometric deep learning model of retinotopy. For more details about the model, please check [here](https://www.sciencedirect.com/science/article/pii/S1053811921008971).

**Requirements:**
- freesurfer directory;
- HCP "fs_LR-deformed_to-fsaverage" surfaces, available [here]( https://github.com/Washington-University/HCPpipelines/tree/master/global/templates/standard_mesh_atlases/resample_fsaverage).

## Requirement 1: Download HCP template surfaces
Create a "templates" (for example, /home/jovyan/templates/) folder and download the following files from the link above:
1. "fs_LR-deformed_to-fsaverage.R.sphere.32k_fs_LR.surf.gii";
2. "fs_LR-deformed_to-fsaverage.L.sphere.32k_fs_LR.surf.gii";


## Requirement 2: FreeSurfer directory
Ensure your FreeSurfer directory is organized as follows:
```
.
├── 102311
│   └── surf
│       ├── lh.pial
│       ├── lh.sphere
│       ├── lh.sphere.reg
│       ├── lh.thickness
│       ├── lh.white
│       ├── rh.graymid
│       ├── rh.graymid.H
│       ├── rh.graymid.K
│       ├── rh.pial
│       ├── rh.sphere
│       ├── rh.sphere.reg
│       ├── rh.thickness
│       └── rh.white
├── 102816
│   └── surf
│       └── ...
```

We use the midthickness surface instead of the white or pial surfaces in the current prediction pipeline. Given that generating the midthickness surface with FreeSurfer takes up to 2 hours, we provide the midthickness surfaces to speed up retinotopic map generation.

## Setting up Neurodesk

In [1]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [2]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: GenuineIntel
model name	: Intel(R) Xeon(R) Platinum 8488C


## Predicting retinotopy maps with deepRetinotopy

In [3]:
# load deepretinotopy
import lmod
import os
await lmod.load('deepretinotopy/1.0.2')
await lmod.list()

['deepretinotopy/1.0.2']

In [4]:
!which python

/cvmfs/neurodesk.ardc.edu.au/containers/deepretinotopy_1.0.2_20231223/python


In [5]:
%%bash
date_tag=20231223
cd /cvmfs/neurodesk.ardc.edu.au/containers/deepretinotopy_1.0.2_"$date_tag"/deepretinotopy_1.0.2_$date_tag.simg/opt/deepRetinotopy_TheToolbox/
bash deepRetinotopy.sh \
    -s /home/jovyan/shared/lec-03/data/freesurfer \
    -t /home/jovyan/shared/templates/ \
    -d hcp \
    -m "polarAngle,eccentricity"

tee: deepRetinotopy_output.log: Read-only file system


Path subs directory: /home/jovyan/shared/lec-03/data/freesurfer
Path to fs_LR-deformed_to-fsaverage surfaces: /home/jovyan/shared/templates/
Dataset name: hcp
Maps: polarAngle eccentricity
Generating mid-thickness surface and curvature data...


tee: deepRetinotopy_error.log: Read-only file system


Path: /home/jovyan/shared/lec-03/data/freesurfer
Hemisphere: lh
Subject: sub-wlsubj001
Generating mid-thickness surface and curvature data if not available...
using distance as a % of thickness
expanding surface sub-wlsubj001/surf/lh.white by 50.0% of thickness and writing it to sub-wlsubj001/surf/lh.graymid
reading thickness...
min=0.000, parallel=0.0078, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0076, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0319, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0319, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0325, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0607, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.2425, normal=0.0000, spring=0.0000, ashburner=0.000, tsmooth=0.000
min=0.000, parallel=0.0316, normal=0.0000, spr

total integrated curvature = 20.324*4pi (255.398) --> -19 handles
ICI = 148.8, FI = 1251.0, variation=20484.148
writing Gaussian curvature to sub-wlsubj001/surf/lh.graymid.K...done.
writing mean curvature to sub-wlsubj001/surf/lh.graymid.H...done.


...

Preparing native surfaces for resampling if not available...
Saving sub-wlsubj001/surf/rh.midthickness.surf.gii.temp10810.white.surf.gii as a surface
Saving sub-wlsubj001/surf/rh.midthickness.surf.gii.temp10810.pial.surf.gii as a surface
Saving sub-wlsubj001/surf/rh.sphere.reg.surf.gii as a surface
Resampling native to fsaverage space...
Execution time: 13 minutes
Retinotopy prediction...


/opt/miniconda-py37_4.8.3/lib/python3.7/site-packages/torch_geometric/nn/conv/spline_conv.py:118: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  'We do not recommend using the non-optimized CPU version of '


Loading the dataset
Processing...
Done!
Dataset loaded
Time elapsed: 0.24621056765317917 minutes
Loading model 1
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 1 were saved
Time elapsed: 2.00151639059186 minutes
Loading model 2
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 2 were saved
Time elapsed: 1.8989524021744728 minutes
Loading model 3
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 3 were saved
Time elapsed: 1.892270315438509 minutes
Loading model 4
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 4 were saved
Time elapsed: 1.9009521342813969 minutes
Loading model 5
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 5 were saved
Time elapsed

/opt/miniconda-py37_4.8.3/lib/python3.7/site-packages/torch_geometric/nn/conv/spline_conv.py:118: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  'We do not recommend using the non-optimized CPU version of '


Loading the dataset
Processing...
Done!
Dataset loaded
Time elapsed: 0.2451774887740612 minutes
Loading model 1
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 1 were saved
Time elapsed: 1.957329899072647 minutes
Loading model 2
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 2 were saved
Time elapsed: 1.902265939861536 minutes
Loading model 3
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 3 were saved
Time elapsed: 1.8866022191941738 minutes
Loading model 4
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 4 were saved
Time elapsed: 1.88389952480793 minutes
Loading model 5
Saving .gii files in: /home/jovyan/shared/lec-03/data/freesurfersub-wlsubj001/deepRetinotopy/
Predictions from model 5 were saved
Time elapsed: 